<h1> EC558 Project Core Pipeline</h1>

<h2> 1: Data Loading and Initial Inspection </h2>

<h3>1.1: Imports and Basic Setup</h3>

In [2]:
import os
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None) 

DATA_DIR = "../data" 
RAW_FILE = os.path.join(DATA_DIR, "historicdata_15min_6months_raw.csv"

<h3>1.2: Load Raw Dataset </h3>

In [3]:
df_raw = pd.read_csv(RAW_FILE) 
print("Shape (rows, columns):", df_raw.shape)
df_raw.head()

Shape (rows, columns): (4418, 18)


,Date Time,Date Time(RAW),Traffic Total (volume),Traffic Total (volume)(RAW),Traffic Total (speed),Traffic Total (speed)(RAW),Traffic In (volume),Traffic In (volume)(RAW),Traffic In (speed),Traffic In (speed)(RAW),Traffic Out (volume),Traffic Out (volume)(RAW),Traffic Out (speed),Traffic Out (speed)(RAW),Downtime,Downtime(RAW),Coverage,Coverage(RAW)
0,3/1/2025 12:00:00 AM - 1:00:00 AM,45716.95833,"331,414,316 KByte",3.390000e+11,"752,542 kbit/s",94067760.37,"34,990,821 KByte",3.583060e+10,"79,454 kbit/s",9931701.854,"296,423,494 KByte",3.040000e+11,"673,088 kbit/s",84136058.51,0%,0.0,100%,10000.0
1,3/1/2025 1:00:00 AM - 2:00:00 AM,45717.00000,"309,228,336 KByte",3.170000e+11,"703,723 kbit/s",87965368.47,"33,772,629 KByte",3.458317e+10,"76,858 kbit/s",9607210.576,"275,455,708 KByte",2.820000e+11,"626,865 kbit/s",78358157.90,0%,0.0,100%,9999.0
2,3/1/2025 2:00:00 AM - 3:00:00 AM,45717.04167,"274,829,321 KByte",2.810000e+11,"625,443 kbit/s",78180405.69,"29,956,768 KByte",3.067573e+10,"68,174 kbit/s",8521770.157,"244,872,552 KByte",2.510000e+11,"557,269 kbit/s",69658635.54,0%,0.0,100%,9999.0
3,3/1/2025 3:00:00 AM - 4:00:00 AM,45717.08333,"219,381,312 KByte",2.250000e+11,"499,257 kbit/s",62407169.45,"23,546,502 KByte",2.411162e+10,"53,586 kbit/s",6698248.380,"195,834,811 KByte",2.010000e+11,"445,671 kbit/s",55708921.07,0%,0.0,100%,9999.0
4,3/1/2025 4:00:00 AM - 5:00:00 AM,45717.12500,"169,163,809 KByte",1.730000e+11,"384,971 kbit/s",48121314.89,"22,602,400 KByte",2.314486e+10,"51,437 kbit/s",6429609.299,"146,561,410 KByte",1.500000e+11,"333,534 kbit/s",41691705.60,0%,0.0,100%,9999.0


<h3>1.3: Basic Info About Columns</h3>

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4418 entries, 0 to 4417
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date Time                    4418 non-null   object 
 1   Date Time(RAW)               4416 non-null   float64
 2   Traffic Total (volume)       4418 non-null   object 
 3   Traffic Total (volume)(RAW)  4416 non-null   float64
 4   Traffic Total (speed)        4417 non-null   object 
 5   Traffic Total (speed)(RAW)   4416 non-null   float64
 6   Traffic In (volume)          4418 non-null   object 
 7   Traffic In (volume)(RAW)     4416 non-null   float64
 8   Traffic In (speed)           4417 non-null   object 
 9   Traffic In (speed)(RAW)      4416 non-null   float64
 10  Traffic Out (volume)         4418 non-null   object 
 11  Traffic Out (volume)(RAW)    4416 non-null   float64
 12  Traffic Out (speed)          4417 non-null   object 
 13  Traffic Out (speed

<h3>1.4: Stats for numeric Columns</h3>

In [5]:
df_raw.describe()

,Date Time(RAW),Traffic Total (volume)(RAW),Traffic Total (speed)(RAW),Traffic In (volume)(RAW),Traffic In (speed)(RAW),Traffic Out (volume)(RAW),Traffic Out (speed)(RAW),Downtime(RAW),Coverage(RAW)
count,4416.000000,4.416000e+03,4.416000e+03,4.416000e+03,4.416000e+03,4.416000e+03,4.416000e+03,4416.000000,4416.000000
mean,45808.937500,2.697381e+11,7.519989e+07,2.622148e+10,7.308459e+06,2.435120e+11,6.789143e+07,0.568963,9994.289629
std,53.122238,9.499039e+10,2.635654e+07,8.452537e+09,2.339738e+06,8.774713e+10,2.435466e+07,6.694339,29.901986
min,45716.958330,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,9348.000000
25%,45762.947915,1.830000e+11,5.135180e+07,1.994645e+10,5.583655e+06,1.660000e+11,4.636938e+07,0.000000,9999.000000
50%,45808.937500,2.910000e+11,8.104201e+07,2.687048e+10,7.481708e+06,2.630000e+11,7.346625e+07,0.000000,9999.000000
75%,45854.927085,3.450000e+11,9.624629e+07,3.230263e+10,8.999091e+06,3.140000e+11,8.734059e+07,0.000000,9999.000000
max,45900.916670,4.420000e+11,1.227932e+08,5.279937e+10,1.470660e+07,4.040000e+11,1.123344e+08,100.000000,10000.000000


<h3>1.5: Column List</h3>

In [6]:
df_raw.columns.tolist()

['Date Time',
 'Date Time(RAW)',
 'Traffic Total (volume)',
 'Traffic Total (volume)(RAW)',
 'Traffic Total (speed)',
 'Traffic Total (speed)(RAW)',
 'Traffic In (volume)',
 'Traffic In (volume)(RAW)',
 'Traffic In (speed)',
 'Traffic In (speed)(RAW)',
 'Traffic Out (volume)',
 'Traffic Out (volume)(RAW)',
 'Traffic Out (speed)',
 'Traffic Out (speed)(RAW)',
 'Downtime',
 'Downtime(RAW)',
 'Coverage',
 'Coverage(RAW)']

In [7]:
df_raw["Date Time"].head()

0    3/1/2025 12:00:00 AM - 1:00:00 AM
1     3/1/2025 1:00:00 AM - 2:00:00 AM
2     3/1/2025 2:00:00 AM - 3:00:00 AM
3     3/1/2025 3:00:00 AM - 4:00:00 AM
4     3/1/2025 4:00:00 AM - 5:00:00 AM
Name: Date Time, dtype: object

In [8]:
df_raw["Date Time(RAW)"].head()

0    45716.95833
1    45717.00000
2    45717.04167
3    45717.08333
4    45717.12500
Name: Date Time(RAW), dtype: float64

In [9]:
df_raw["Traffic Total (speed)"].head()

0    752,542 kbit/s
1    703,723 kbit/s
2    625,443 kbit/s
3    499,257 kbit/s
4    384,971 kbit/s
Name: Traffic Total (speed), dtype: object

In [10]:
df_raw["Traffic Total (speed)(RAW)"].head()

0    94067760.37
1    87965368.47
2    78180405.69
3    62407169.45
4    48121314.89
Name: Traffic Total (speed)(RAW), dtype: float64

<h3>1.6: Checking Missing Values</h3>

In [16]:
null_counts = df_raw.isna().sum()
null_counts[null_counts > 0].sort_values(ascending=False)

Downtime                       4
Date Time(RAW)                 2
Traffic Total (volume)(RAW)    2
Traffic Total (speed)(RAW)     2
Traffic In (volume)(RAW)       2
Traffic In (speed)(RAW)        2
Traffic Out (volume)(RAW)      2
Traffic Out (speed)(RAW)       2
Downtime(RAW)                  2
Coverage(RAW)                  2
Traffic Total (speed)          1
Traffic In (speed)             1
Traffic Out (speed)            1
Coverage                       1
dtype: int64

In [17]:
null_pct = (df_raw.isna().sum() / len(df_raw)) * 100
null_pct[null_pct > 0].sort_values(ascending=False)

Downtime                       0.090539
Date Time(RAW)                 0.045269
Traffic Total (volume)(RAW)    0.045269
Traffic Total (speed)(RAW)     0.045269
Traffic In (volume)(RAW)       0.045269
Traffic In (speed)(RAW)        0.045269
Traffic Out (volume)(RAW)      0.045269
Traffic Out (speed)(RAW)       0.045269
Downtime(RAW)                  0.045269
Coverage(RAW)                  0.045269
Traffic Total (speed)          0.022635
Traffic In (speed)             0.022635
Traffic Out (speed)            0.022635
Coverage                       0.022635
dtype: float64

<h3>1.7: Check for Fully Duplicated Rows</h3>

In [18]:
num_dupes = df_raw.duplicated().sum()
print("Number of fully duplicated rows:", num_dupes)

Number of fully duplicated rows: 0


<h3> 1.8: Summary of Findings</h3>

<p>
 
- Dataset shape: approximately `[4418]` rows and `[18]` columns.
    
- Key time columns:
    
  - `Date Time` (string interval).
    
  - `Date Time(RAW)` (numeric timestamp).
    
- Key traffic columns identified:
    
  - `Traffic Total (speed)` and `Traffic Total (speed)(RAW)`.
    
  - `Traffic In (volume)(RAW)`, `Traffic Out (volume)(RAW)`, etc.
    
- Missing values:
    
Downtime                       4
    
Date Time(RAW)                 2
    
Traffic Total (volume)(RAW)    2
    
Traffic Total (speed)(RAW)     2
    
Traffic In (volume)(RAW)       2
    
Traffic In (speed)(RAW)        2
    
Traffic Out (volume)(RAW)      2
    
Traffic Out (speed)(RAW)       2
    
Downtime(RAW)                  2
    
Coverage(RAW)                  2
    
Traffic Total (speed)          1
    
Traffic In (speed)             1
    
Traffic Out (speed)            1
    
Coverage                       1
    
- Duplicated rows: No duplicated rows found
    
</p>